# Исследование лояльности пользователей телекоммуникационной компании.

**Описание**

Компания провела опрос и попросила меня подготовить дашборд с его итогами. 
Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Чтобы оценить результаты опроса, оценки поделим на группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

*Задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России*


**Процесс выполнения задачи**
- Подключение к базе данных
- Извлечение необходимых данных
- Анализ данных в Tableau

**Описание данных**

[Скачать БД](https://code.s3.yandex.net/datasets/telecomm_csi.db)

Таблица user
- Содержит основную информацию о клиентах.
  - user_id	Идентификатор клиента, первичный ключ таблицы
  - lt_day	Количество дней «жизни» клиента
  - age	Возраст клиента в годах
  - gender_segment	Пол клиента (1 – женщина, 0 – мужчина)
  - os_name	Тип операционной системы
  - cpe_type_name	Тип устройства
  - location_id	Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
  - age_gr_id	Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
  - tr_gr_id	Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице  traffic_segment
  - lt_gr_id	Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
  - nps_score	Оценка клиента в NPS-опросе (от 1 до 10)
  
Таблица location
- Справочник территорий, в которых телеком-компания оказывает услуги.
  - location_id	Идентификатор записи, первичный ключ
  - country	Страна
  - city	Город
  
Таблица age_segment
- Данные о возрастных сегментах клиентов.
  - age_gr_id	Идентификатор сегмента, первичный ключ
  - bucket_min	Минимальная граница сегмента
  - bucket_max	Максимальная граница сегмента
  - title	Название сегмента
  
Таблица traffic_segment
- Данные о выделяемых сегментах по объёму потребляемого трафика.
  - tr_gr_id	Идентификатор сегмента, первичный ключ
  - bucket_min	Минимальная граница сегмента
  - bucket_max	Максимальная граница сегмента
  - title	Название сегмента
  
Таблица lifetime_segment
- Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.
  - lt_gr_id	Идентификатор сегмента, первичный ключ
  - bucket_min	Минимальная граница сегмента
  - bucket_max	Максимальная граница сегмента
  - title	Название сегмента

In [1]:
#!pip install SQLAlchemy==1.4.0
#!pip install --upgrade 'sqlalchemy<2.0'

import pandas as pd
import os
from sqlalchemy import create_engine, text

Для создания дашборда нам понадобятся
- user_id	Идентификатор клиента
- lt_day	Количество дней «жизни» клиента
- is_new	Поле хранит информацию о том, является ли клиент новым
- age	Возраст
- gender_segment	Пол (для удобства работы с полем преобразуйте значения в текстовый вид)
- os_name	Тип операционной системы
- cpe_type_name	Тип устройства
- country	Страна проживания
- city	Город проживания
- age_segment	Возрастной сегмент
- traffic_segment	Сегмент по объёму потребляемого трафика
- lifetime_segment	Сегмент по количеству дней «жизни»
- nps_score	Оценка клиента в NPS-опросе
- nps_group	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [2]:
# Путь к БД на вашем ПК
path_to_db_local = 'telecomm_csi.db'
# Путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# Итоговый путь к БД
path_to_db = None

# Если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# Иначе: если путь на плаформе ведет к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# Иначе выведится сообщение о том, что файл не найден 
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# Если итоговый путь не пустой
if path_to_db:
    # То создаем подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    


In [ ]:
# Запрос
    query = """
SELECT 
    u.user_id AS id,
    u.lt_day,
    u.nps_score,
    u.age,
    u.os_name,
    u.cpe_type_name,
    loc.country AS country,
    loc.city AS city,
    age.title AS age_segment,
    tr.title AS traffic_segment,
    lt.title AS lifetime_segment,
    CASE
        WHEN u.lt_day < 365 THEN 'new'
        ELSE 'old'
    END AS age_group,
    CASE
        WHEN u.gender_segment = 1 THEN 'woman'
        ELSE 'man'
    END AS sex,
    CASE
        WHEN u.nps_score IN (10,9) THEN 'promoters'
        WHEN u.nps_score IN (8,7) THEN 'passives'
        ELSE 'detractors'
    END AS nps_goup
FROM user AS u
LEFT JOIN location AS loc ON u.location_id=loc.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id=age.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id=tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
"""

In [ ]:
# Создаем датафрейм по данным запроса
df = pd.read_sql(query, engine)

In [3]:
# Проверка данных
df.head()

,id,lt_day,nps_score,age,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,age_group,sex,nps_goup
0,A001A2,2320,10,45.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,old,woman,promoters
1,A001WF,2344,10,53.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,old,man,promoters
2,A003Q7,467,10,57.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,old,man,promoters
3,A004TB,4190,10,44.0,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,old,woman,promoters
4,A004XT,1163,10,24.0,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,old,man,promoters


In [4]:
# Сохранение файла
#df.to_csv('users_data.csv', index=False)

Презентация состоит из 2х слайдов.
- На первом слайде основной анализ по запросу заказчика.
- На втором слайде дополнительный анализ по аномальным данным.

Для более комфортного просмотра рекомендуется переключить перезентацию в полноэкранный режим.

[TableauDashboard](https://public.tableau.com/views/Users_17088539109290/__2?:language=en-US&:sid=&:display_count=n&:origin=viz_share_link)
<a id='DashBoard'></a>
